In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import pygame

import torch
import torch.nn as nn
import torch.nn.functional as F

from math import *
from IPython import display

import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
im = np.array(Image.open("/home/nichg/Dropbox/Pictures/IMG_0822.JPG"))
im = im[::8,::8]

print(im.shape)

(324, 432, 3)


In [3]:
XRES = 256
YRES = 256
MULT = 1

pygame.init()
screen = pygame.display.set_mode((XRES, YRES), 0, 24)

In [14]:
def getSequences(t):
    theta1 = 2*t/659101
    theta2 = t/329663
    
    coord = np.concatenate([XRES/2 + (XRES/2)*np.cos(theta1)[:,np.newaxis], YRES/2 + (YRES/2)*np.cos(theta2)[:,np.newaxis]], axis=1)
    coord = coord.astype(np.uint32)
    coord[:,0] = np.clip(coord[:,0], 0, YRES-1)
    coord[:,1] = np.clip(coord[:,1], 0, XRES-1)
    
    rgb = im[coord[:,0], coord[:,1]]
    rgb = np.concatenate([rgb, 0.9*np.ones((rgb.shape[0], 1))], axis=1)
    return rgb, theta1, theta2

def render(theta1, theta2, rgb):
    view = np.zeros((XRES, YRES, 4))
    
    coord = np.concatenate([XRES/2 + (XRES/2)*np.cos(theta1)[:,np.newaxis], YRES/2 + (YRES/2)*np.cos(theta2)[:,np.newaxis]], axis=1)
    coord = coord.astype(np.uint32)
    coord[:,0] = np.clip(coord[:,0], 0, YRES-1)
    coord[:,1] = np.clip(coord[:,1], 0, XRES-1)
    
    view[coord[:,1], coord[:,0]] = rgb
    
    return view

In [15]:
!mkdir output2
!rm output2/*.png

t0 = 0
frame = 0
view = np.zeros((XRES,YRES,3))
glitch = 0

with torch.no_grad():
    while True:
        for event in pygame.event.get():
            pass
        
        im = np.array(Image.open("/home/nichg/Dropbox/art/logo/%.4d.png" % (1+frame%100)))[:,:,:3]
        
        t = 100*np.arange(400000) + t0
        rgb, theta1, theta2 = getSequences(t)
        ofsrgb, ofstheta1, ofstheta2 = getSequences(t + np.random.randint(2000)-1000)
        
        if np.random.rand()<0.01:
            glitch = 5 + np.random.randint(10)
            
        if glitch>0:
            #rgb = ofsrgb
            theta1 = ofstheta1
            theta2 = ofstheta2
            #theta1 = np.roll(theta1, np.random.randint(11)-5)
            #theta2 = np.roll(theta2, np.random.randint(11)-5)
            glitch -= 1
        
        t0 += 40000000
        
        r = render(theta1, theta2, rgb)
        view = np.clip((0.999*view*(1-r[:,:,3:]) + 1.4*r[:,:,:3]*r[:,:,3:]),0,255)
        im = Image.fromarray(view.astype(np.uint8).transpose(1,0,2))
        im.save("output2/%.6d.png" % frame)
        s_image = pygame.pixelcopy.make_surface(view.astype(np.uint8))

        screen.blit(s_image, (0, 0))

        pygame.display.update()   
        time.sleep(0.005)
        
        frame += 1

rm: cannot remove 'output2/*.png': No such file or directory


KeyboardInterrupt: 